# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

# Floating-Point Arithmetic

The default floating-point number system is **Float64**, which is double precision on **MATLAB**. Let us find the value of machine epsilon and range as follows.

In [ ]:
eps()

In [ ]:
floatmax()

In [ ]:
floatmin()

# Basics of Rounding

## Types of functions:

[1] **ceil(x)** returns the next largest integer to x (towards + ∞).

[2] **floor(x)** returns the next smallest integer to x (towards - ∞).

[3] **trunc(x)** returns the closest integer towards zero (round to zero).

[4] **round(x,mode)** function with several modes options, see: https://docs.julialang.org/en/v1/base/math/#Base.round-Tuple{Type,%20Any}


In [ ]:
ceil(1.3)

In [ ]:
floor(1.3)

In [ ]:
trunc(1.3)

In [ ]:
ceil(-1.3)

In [ ]:
floor(-1.3)

In [ ]:
trunc(-1.3)

In [ ]:
round(1.3) ## round to nearest by default

In [ ]:
round(-1.3) ## round to neart by default

In [ ]:
round(1.3,RoundToZero)

In [ ]:
round(-1.3,RoundToZero)

In [ ]:
round(1.3,RoundUp)

In [ ]:
round(1.3,RoundDown)

In [ ]:
round(-1.3,RoundUp)

In [ ]:
round(-1.3,RoundDown)

# Stochastic Rounding

Not included in Julia by deafult, a package is available called StochasticRounding.jl

I used a popular randomly round to nearest integer idea as formed in  https://gist.github.com/coleoguy/a50dba496fc55440cb4945fe8e25d703 as mentioned in https://medium.com/@minghz42/what-is-stochastic-rounding-b78670d0c4a to create a version for it in Julia.

In [ ]:
using Pkg
Pkg.add("Distributions")

In [ ]:
using Distributions
function StochasticRound(x)
    # extract the decimal portion
    r=trunc(x)
    q=abs(x-r)
    # draw a value 0 or 1 with probability
    # based on how close we already are
    Δx = wsample(0:1,[1-q,q],1)
     # make it negative if x is negative
     if(x<0)
         Δx=-Δx
     end
     # return our new value
     r+Δx[1]
end

In [ ]:
StochasticRound(1.3)

In [ ]:
StochasticRound(1.3)

## How to "see" the probability?

Let us plot it! 



In [ ]:
using Plots

global ℘=0
n=10000
print("Enter x:")
x = readline()
x=parse(Float64,x)
for i=1:n
    result=StochasticRound(x)
    if result==ceil(x)
        global ℘=℘+1
    end
end
global ℘=℘/n
Plots.gr() # the default GR backend look for pie charts
x = ["ceil(x)", "floor(x)"]
y = [℘,1-℘]
pie(x, y, title="Stochastic Rounding", l=0.5)